Az osztály(oka)t mentsd a __src/linear_regression__ modul __LinearRegressions.py__ fájljába
Használható modulok: _pathlib, pandas, typing, str, numpy_, valamint a _scipy.stats t_ és _f_ osztályai

In [2]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

In [3]:
from pathlib import Path

<IPython.core.display.Javascript object>

In [4]:
datalib = Path.cwd().parent.joinpath('data')

<IPython.core.display.Javascript object>

### Adatelőkészítés (0 pont)
1., Olvasd be a data mappa __sp500.parquet__ nevű fájlját egy DataFrame-be. A betöltéshez használt engine paramétere legyen <u>fastparquet</u>
2., Olvasd be az __ff_factors.parquet__ fájlt egy DataFrame-be. A betöltéshez használt engine paramétere legyen <u>fastparquet</u>
3., Kapcsold össze a két DataFrame-t egy új DataFrame-be. Az összekapcsolás módja, hogy a hozam adatokra balról kapcsoljuk rá a factor adatokat a __'Date' elsődleges kulcs__ alapján.
4., Készíts egy új __'Excess Return'__ nevű oszlopot, ami a havi hozamok és a kockázat mentes hozam (RF) különbsége
5., <u>Rendezd sorba dátum szerint az adatokat</u>, majd generálj egy új oszlopot (__'ex_ret_1'__), amely minden ticker ('Symbol') esetén 1-el eltolja az Excess Return értékeit olyan módon, hogy minden sorban szerepeljen a <u>következő időszaki Excess Return</u> érték. 
6., A meglévő adathalmazt írd felül olyan módon, hogy egyszer <u>törlöd az össze olyan sort</u>, amely az __'ex_ret_1' oszlopban hiányos__, majd ezt követően, törlöd az összes olyan sort, ami a __'HML' oszlopban hiányos__.


3. feladat segítség
![Joined data](../resources/weekly6/joined_data.jpg)

5. feladat segítség
![new column](../resources/weekly6/ex_ret_1.jpg)

In [16]:
import pandas
import pandas as pd
from typing import List, Dict
import matplotlib
import matplotlib.pyplot as plt
from pathlib import Path
import statsmodels.api as sm

sp500 = pd.read_parquet('/Users/kissr/Downloads/Programming/Python/ECOPY_23241/data/sp500.parquet',
                        engine='fastparquet')
ff_factors = pd.read_parquet('/Users/kissr/Downloads/Programming/Python/ECOPY_23241/data/ff_factors.parquet',
                             engine='fastparquet')

# 3
merged_df = pd.merge(sp500, ff_factors, on='Date', how='left')

# 4
merged_df['Excess Return'] = merged_df['Monthly Returns'] - merged_df['RF']

# 5
merged_df = merged_df.sort_values(by=['Symbol', 'Date'])
merged_df['ex_ret_1'] = merged_df.groupby('Symbol')['Excess Return'].shift(-1)

# 6
merged_df = merged_df.dropna(subset=['ex_ret_1'])
merged_df = merged_df.dropna(subset=['HML'])

# 7
amazon_df = merged_df[merged_df['Symbol'] == 'AMZN']
amazon_df = amazon_df.drop(columns=['Symbol'])
amazon_df

y = amazon_df[['ex_ret_1']]
x = amazon_df[['Mkt-RF', 'SMB', 'HML']]
y

,ex_ret_1
Date,
2000-01-31,0.023796
2000-02-29,-0.074223
2000-03-31,-0.174575
2000-05-31,-0.288383
2000-06-30,-0.218396
...,...
2022-05-31,-0.122459
2022-06-30,-0.079615
2022-08-31,-0.127622


### Modell összeállítás (7 pont)

A meglévő adatokból válaszd ki a __Amazon részvényhez tartozó sorokat (AMZN)__ és töröld a tickereket tartalmazó oszlopot. <u>Amennyiben működött, önellenőrzésre használhatod a legutóbbi zárthelyin írt osztályt is.</u>

7., Készíts egy új __LinearRegressionGLS__ elnevezésű osztályt. Definiáld benne a __\_\_init\_\___ nevű függvényt, amely bemenetként 2 DataFrame-t kap amelyeket ments le a left_hand_side és right_hand_side elnevezésű változókba. Az egyik DataFrame fogja tartalmazni a következő hónap többlet hozamait (__left_hand_side__), a másik a piaci hozamokat (Mkt-RF), az SMB és a HML értékeket (__right_hand_side__).

8., Egésztsd ki az osztályt egy __fit__ metódussal, ami Feasible GLS elvű becslést hajt végre. Figyelj oda, hogy a regresszió futtatása során konstans (alfa / béta_0) is szerepeljen a predictor változók között, amely az első változó legyen. <u>__(numpy.linalg.lstsq() nem használható)__</u>

9., Egészítsd ki az osztályt egy __get_params__ metódussal, ami visszaadja a becsült modell béta paramétereinek értékeit. A visszakapott pandas Series típusú adatban az oszlop neve legyen __Beta coefficients__. 

10., Egészítsd ki az osztályt egy __get_pvalues__ metódussal, ami visszaadja a becsült modell paraméterekhez tartozó p értékeket. A visszakapott pandas Series típusú adatban az oszlop neve legyen: __P-values for the corresponding coefficients__. A p értéket t-statisztika alapján számold ki. A p-érték kiszámításánál figyelj alkalmazd a <u>min(value, 1-value) * 2</u> képletet.

11., Egészítsd ki az osztályt egy __get_wald_test_result__ metódussal, ami visszaadja a bemeneti restrikciós mátrix alapján számolt F és p értékeket. A visszatérési típus string legyen, a visszaadandó szöveg: __Wald: wald_value, p-value: p_value__, ahol az wald_value és p_value helyére 3 tizedesjegyre kerekítve (__:.3f__) add meg a hozzájuk tartozó értékeket. A függvény bemenete során feltételezzük, hogy r minden eleme 0, R-t listák listája formában adjuk át. A Wald statisztika p értékét az előző feladathoz hasonlóan számold ki, de figyelj, hogy a <u>teszt 1 oldalú</u>.

12., Egészítse ki az osztályt egy __get_model_goodness_values__ metódussal, ami visszadja a centrált és a módosított R-négyzet értékeket. A visszatérési típus string legyen, a visszaadandó szöveg: __Centered R-squared: crs, Adjusted R-squared: ars__, ahol crs és ars helyére 3 tizedesjegyre kerekítve (__:.3f__) add meg a hozzájuk tartozó értékeket. <u>Ha a regresszorok számába eredetileg beleszámítottad a konstanst is, akkor a módosított R-négyzet számítás nevezőjében nincs szükség a __-1__-es tagra</u>.

__Feasible GLS becslésének a menete__:
1.) A bementeni adatok alapján OLS elven becsüld meg a változók együtthatóit.
2.) Számold ki a hibatagokat
3.) A hibatagoknak vedd a négyzetét
4.) Becsülj egy új modellt, ahol a bal oldali változó a hibatag négyzetének logaritmusa, míg a jobboldali változók az eredeti jobb oldali változóid.
5.) Számold ki a becsült értékeket (logaritmikus négyzetes hibákat) és vedd őket az e kitevőjeként, majd vonj belőlük gyököt.
6.) A kapott vektornak vedd elemenként az inverzét és helyezd el egy diagonális mátrix főátlójában őket (np.diag). -> Az így kapott mátrix lesz a V inverz mátrix.
7.) A V inverz mátrix felhasználásával becsüld meg a GLS regresszió paramétereit
8.) Egyszerűsítés képpen számold ki a hibatagot a nem transzformált Y és X mátrixok és GLS paraméterek segítségével.

__FGLS R^2 számításhoz__:
SSR =  Y\^T * V\^(-1) * X * (X\^T * V\^(-1) * X)\^(-1) * X\^T * V\^(-1) * Y
SST = Y\^T * V\^(-1) * Y

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [18]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import scipy


class LinearRegressionGLS:

    def __init__(self, left_hand_side, right_hand_side):
        self.left_hand_side = left_hand_side
        self.right_hand_side = right_hand_side
        self.right_hand_side.insert(0, 'alfa', 1)
        self._model = None

    def fit(self):
        Y = self.left_hand_side
        X = self.right_hand_side
        self.XtX = np.dot(np.transpose(self.right_hand_side), self.right_hand_side)
        self.XtX_inv = np.linalg.inv(self.XtX)
        self.Xty = np.dot(np.transpose(self.right_hand_side), self.left_hand_side)
        self.betas = np.dot(self.XtX_inv, self.Xty)
        self.residuals = self.left_hand_side - np.dot(self.right_hand_side, self.betas)
        self.residuals = self.residuals**2
        self.residuals = np.log(self.residuals)
        Y_new= self.residuals
        X_new = self.right_hand_side
        self.XtX = np.dot(np.transpose(self.right_hand_side), self.right_hand_side)
        self.XtX_inv = np.linalg.inv(self.XtX)
        self.Xty = np.dot(np.transpose(self.right_hand_side), self.residuals)
        self.betas = np.dot(self.XtX_inv, self.Xty)
        self.residuals_new = self.residuals - np.dot(self.right_hand_side, self.betas)
        self.y_pred = self.right_hand_side@self.betas + self.residuals_new
        self.y_pred = np.exp(self.y_pred)
        self.y_pred = np.sqrt(self.y_pred)
        self.y_pred_inv = np.reciprocal(self.y_pred)
        self.V = np.diag(self.y_pred_inv)

        return

    def get_params(self):
        Y = self.left_hand_side
        X = self.right_hand_side
        self.XtX_gls = self.right_hand_side.T@self.V@self.right_hand_side
        self.XtX_inv_gls = np.linalg.inv(self.XtX_gls)
        self.Xty_gls = self.right_hand_side.T@self.V@self.left_hand_side
        self.betas_gls = self.XtX_inv_gls@self.Xty_gls
        return pd.Series(self.betas, name='Beta coefficients')

    def get_pvalues(self):
        self.residuals_gls = self.left_hand_side - self.right_hand_side@self.betas_gls
        self.n = len(self.left_hand_side)
        self.K = len(self.right_hand_side.columns)
        self.df = self.n - self.K
        self.variance_gls = self.residuals_gls.T@self.residuals_gls / self.df
        self.stderror_gls = np.sqrt(self.variance_gls * np.diag(self.XtX_inv_gls))
        self.t_stat_gls = np.divide(self.betas_gls, self.stderror_gls)
        term = np.minimum(scipy.stats.t.cdf(self.t_stat_gls, self.df), 1 - scipy.stats.t.cdf(self.t_stat_gls, self.df))
        p_values = (term) * 2
        return pd.Series(p_values, name='P-values for the corresponding coefficients')

    def get_wald_test_result(self, restr_matrix):
        term_1 = np.dot(restr_matrix, self.betas)
        term_2 = np.dot(np.dot(restr_matrix, self.XtX_inv_gls), np.transpose(restr_matrix))
        f_stat = (np.dot(np.transpose(term_1), np.dot(np.linalg.inv(term_2), term_1)) / len(
            restr_matrix)) / self.variance_gls
        p_value = (1 - scipy.stats.f.cdf(f_stat, len(restr_matrix), self.df))
        f_stat.astype(float)
        p_value.astype(float)
        return f'Wald: {round(f_stat, 3)}, p-value: {round(p_value, 3)}'

    def get_model_goodness_values(self):
        y_demean = self.left_hand_side - sum(self.left_hand_side) / len(self.left_hand_side)
        SST = self.left_hand_side@np.linalg.inv(self.V)@self.left_hand_side
        SSE = np.dot(np.transpose(self.residuals_gls), self.residuals_gls)
        r2 = round(1 - SSE / SST, 3)
        adj_r2 = round(1 - (self.n - 1) / (self.df) * (1 - r2), 3)
        return f'Centered R-squared: {r2:.3f}, Adjusted R-squared: {adj_r2:.3f}'
    
model = LinearRegressionGLS(y,x)
model.y_pred

ValueError: cannot insert alfa, already exists